# Macroeconomic Theory
## Francesco Franco
### T3 2023 

# Simulation of an OLG in Dynare

-   Dynare is the main software used by macroeconomists at Central Banks to simulate and estimate macro models

-   Mostly used for stochastic business cycle models (we will see it with the next model)

-   Can be used to simulate (and estimate) deterministic long run models like the Ramsey (in dicrete time) and the OLG

- we will learn how to create a .mod file that is then parsed by dynare

## Social Planner OLG model
Always start with your model $$U=\lambda_{-1}\beta u(c_{20})+\sum_{t=0}^{\infty}\lambda_{t}\left(u(c_{1t})+\beta u(c_{2t+1})\right)$$

The resource constraint of the planner is with multiplier $\kappa_{t}$ $$k_{t-1}+z_tf(k_{t-1})=(1+n)k_{t}+c_{1t}+(1+n)^{-1}c_{2t}$$

The timing assumption is to allow the computer to distinguish states and jumps.

### The .mod file

Declare the endogenous variables, here consumption of the young, of the old and capital

    var c1 c2 k;
    
Declare the exogenous variables, here the zt

    varexo z;


## Parametric assumptions

On the computer you need to assume functional forms. Use $u=ln(c)$ and $f=zk^{\alpha}$. Also define $\psi=\frac{\lambda_{t}}{\lambda_{t-1}}$

Declare the parameters

    parameters alpha bet n R psi rho;
    T = 25;
    alpha = 0.3;
    rho = (1 + 0.04)^T - 1;
    bet = 1/(1 + rho);
    n = (1 + 0.01)^T - 1;
    R = (1 + 0.02)^T -1;
    psi = 1/(1 + R);



## Model

The foc conditions $$\frac{\lambda_{t}}{c_{1t}}=\kappa_{t}$$ $$\beta\frac{\lambda_{t-1}}{c_{2t}}=\kappa_{t}(1+n)^{-1}$$ $$\kappa_{t+1}\left(1+z_{t+1}\alpha k_{t}^{\alpha-1}\right)=\kappa_{t}(1+n)$$


## Model in the .mod

combining the foc we obtain the model dynamics of the model $$c_{2t}=\frac{\beta}{\psi}(1+n)c_{1t}$$ $$\frac{c_{t+1}}{c_{1t}}=\frac{\psi}{(1+n)}\left(1+z_{t+1}\alpha k_{t}^{\alpha-1}\right)$$ $$z_{t}k_{t-1}^{\alpha}=(1+n)k_{t}+c_{1t}+(1+n)^{-1}c_{2t}$$

Declare the model

    model;
    k(-1) + z*k(-1)^alpha = (1+n)*k + c1 +c2/(1+n);
    c2 = bet/psi*(1+n)*c1;
    c1(+1)/c1 = psi/(1+n)*(1 + z(+1)*alpha*k(-1)^(alpha-1));
    end;


## The steady state

While the algorithm to solve the determinitic infinite horizon models looks for the global solution it checks that the properties of the dynamic system around the end point are a saddle by linearizing the model around the end point, which is the steady state:

$$k=\left(\frac{z\alpha\psi}{1+n-\psi}\right)^{\frac{1}{1-\alpha}}$$ $$c_{2}=\frac{\beta}{\psi}(1+n)c_{1}$$ $$zk^{\alpha}-nk=\frac{\psi+\beta}{\psi}c_{1}$$ $$c_{1}=\frac{\psi}{\psi+\beta}\left(zk^{\alpha}-nk\right)$$

## The steady state in .mod

Declare the steady state

    initval;
    z = 1;
    k = (psi*z*alpha/(1+n-psi))^(1/(1-alpha));
    c1 = psi/(psi+bet)*(z*k^alpha-n*k);
    c2 = c1*bet*(1+n)/psi;
    end;
    
and check the saddle condition

// Check that this is indeed the steady state

    steady;

// Check the Blanchard-Kahn conditions
    
    check;

## Simulate the model

We are going to increase z by 10% for 80 periods (500 years according to our calibration)
The solution is a perfect foresight solution so the SP knows that in period 81 z goes back to 1.

    shocks;
    var z;
    periods 1:80;
    values 1.1;
    end;
    
    perfect_foresight_setup(periods=100);
    perfect_foresight_solver;
    rplot k;

## Graphical Solution

The solution shows that for a long time the optimal path is to stay close to the steady state value with z=1.1. The increase in k that anticipate period 81 decrease in z is due to the fowrad looking aspect of the solution. The solver cannot solve models with finite horizon you would have to progam it. Still this is a nice illustration of the Turnpike property.

## Overlapping generations with perpetual youth in discrete time 
-   Instead of finite lives constant probability of death $\text{\ensuremath{\nu}}$

-   Individual are potentially infinitely lived but their life will come to an end. The expected utility is therefore $$E_{t}\left[\sum_{t=0}^{\infty}\beta^{t}u\left(c_{t}\right)\right]$$ where the expectations is about the probability of diying, then $$E_{t}\left[\sum_{t=0}^{\infty}\beta^{t}u\left(c_{t}\right)\right]=\sum_{t=0}^{\infty}\left(\beta\left(1-\nu\right)\right)^{t}u\left(c_{t}\right)$$

   Expected life is $\frac{1}{\nu}$ for evry individual that has survived until then, independently of age hence the name perpetual youth


## Individual budget constraint

The individual $i$ budget constraint is $$a_{it+1}=\left(1+r_{t}\right)a_{it}-c_{it}+w_{t}+z_{it}$$ where $z_{it}$ is a transfer. The transfer is present because individuals that die leave their assets behind and we need an assumption on how to redistribute them. For example they could leave them as bequests to the next generation or the government could seize them and redistribute them to the living.



## Annuity markets {#annuity-markets .unnumbered}

An alternative is to introduce annuity markets that consists in financial intermediaries that make a payment during the life of an agent in exchange of receiving their wealth when they die. This is kind of a revere mortgage. We assume perfect competition in the financial intermediation sector. The profit of a financial intermediary on an individual holding asstes $a_{t}$is $$\pi(a_{t})=\nu a_{t}-\left(1-\nu\right)z(a_{t})$$ perfect competition implies that $$z(a_{t})=\frac{\nu}{\left(1-\nu\right)}a_{t}$$


## Individuals indexed by generations

We can now index the individuals $i$ with $\tau$ the year of their birth. The busdget constraint is therefore $$a_{t+1,\tau}=\left(1+r_{t}+\frac{\nu}{\left(1-\nu\right)}\right)a_{t,\tau}-c_{t,\tau}+w_{t}$$ and the FOC $$u'\left(c_{t,\tau}\right)=\beta\left(1-\nu\right)\left(1+r_{t+1}+\frac{\nu}{\left(1-\nu\right)}\right)u'\left(c_{t+1,\tau}\right)$$ which makes the Euler condition dependent on $\nu$. This dependency is present because of the discrete time assumption. $$\left(1+r_{t+1}\right)\left(1-\nu\right)+\nu=\left(1+r_{t+1}\right)-r_{t+1}\nu$$ in continuous time $r_{t+1}\nu=0$.



## Demography 
We assume that each period a number $n$ of new individuals enter the economy: $$L_{t+1}=L_{t}\left(1+n-\nu\right)$$

Therefore at time $t=1$ the number individuals with age 1 is $L_{0}\left(1+n-\nu\right)\left(1-\nu\right)$. At time $t=2$ the number individuals with age 1 is $L_{1}\left(1+n-\nu\right)\left(1-\nu\right)$ and of age 2 is $L_{0}\left(1+n-\nu\right)\left(1-\nu\right)^{2}$. We can then write $$L_{t,\tau}=L_{\tau}\left(1+n-\nu\right)\sum_{t=0}^{t-1}\left(1-\nu\right)^{t}=L_{0}\left(1+n-\nu\right)^{t-\tau}\frac{1-\left(1-\nu\right)^{t}}{\nu}$$


## Aggregation by birth

The aggregation should be as follows $$A_{t,\tau}=a_{t,\tau}L_{t,\tau}$$

$$A_{t}=\sum_{\tau=-\infty}^{t}A_{t,\tau}$$ where the $-\infty$ is there to include all cohort also thos born in the distant past. so let us try with the BC $$a_{t+1,\tau}=\left(1+r_{t}+\frac{\nu}{\left(1-\nu\right)}\right)a_{t,\tau}-c_{t,\tau}+w_{t}$$

multiply by $L_{t,\tau}$ and using $L_{t+1,\tau}\frac{L_{t,\tau}}{L_{t+1,\tau}}=\frac{L_{t+1,\tau}}{\left(1-\nu\right)}$ $$\frac{A_{t+1,\tau}}{\left(1-\nu\right)}=\left(1+r_{t}+\frac{\nu}{\left(1-\nu\right)}\right)A_{t,\tau}-C_{t,\tau}+w_{t}L_{t,\tau}$$

you can aggregate by year of birth.

## Aggregation 

Now aggregate across all $\tau$ and use the fact that $$A_{t+1}=\sum_{\tau=0}^{t+1}A_{t+1,\tau}=\sum_{\tau=0}^{t}A_{t+1,\tau}+a_{t+1,t+1}L_{t+1,t+1}$$

so aggregation is possible (namely expressing in terms of only aggregate variables) if $a_{t+1,t+1}=0$ or of individuals are born without wealth (or bequests). Otherwise micro variables stay in the expression. $$\frac{A_{t+1}}{\left(1-\nu\right)}=\left(1+r_{t}+\frac{\nu}{\left(1-\nu\right)}\right)A_{t}-C_{t}+w_{t}L_{t}$$



## Aggregation of individual conditions
Now aggregate Euler and assume a log utility $$\frac{L_{t,\tau}}{L_{t+1,\tau}}L_{t+1,\tau}c_{t+1,\tau}=\beta\left(1-\nu\right)\left(1+r_{t+1}+\frac{\nu}{\left(1-\nu\right)}\right)c_{t,\tau}L_{t,\tau}$$ $$C_{t+1,\tau}=\beta\left(1-\nu\right)^{2}\left(1+r_{t+1}+\frac{\nu}{\left(1-\nu\right)}\right)C_{t,\tau}$$

but $$C_{t+1}=\sum_{\tau=0}^{t+1}C_{t+1,\tau}=\sum_{\tau=0}^{t}C_{t+1,\tau}+c_{t+1,t+1}L_{t+1,t+1}$$

that is tough! Would not appear in continous time.

## Aggregation of individual conditions 

To be able to aggreagate the Euler condition and define $R_{t}=\left(1+r_{t}+\frac{\nu}{\left(1-\nu\right)}\right)$the Euler is $$c_{t+1,\tau}=\beta\left(1-\nu\right)\left(1+r_{t+1}+\frac{\nu}{\left(1-\nu\right)}\right)c_{t,\tau}$$ and the BC is $$a_{t+1,\tau}=R_{t}a_{t,\tau}-c_{t,\tau}+w_{t}$$ $$a_{t,\tau}=\frac{c_{t,\tau}-w_{t}}{R_{t}}+\frac{a_{t+1,\tau}}{R_{t}}$$ $$a_{t,\tau}=\frac{c_{t,\tau}-w_{t}}{R_{t}}+\frac{c_{t+1,\tau}-w_{t+1}}{R_{t}R_{t+1}}+\frac{a_{t+2,\tau}}{R_{t}R_{t+1}}$$ which sums to $$\sum_{s=t}^{\infty}\frac{c_{s,\tau}}{\prod_{j}^{s}R_{j}}=a_{t,\tau}+\sum_{s=t}^{\infty}\frac{w_{s,\tau}}{\prod_{j}^{s}R_{j}}$$



## Aggregation of individual conditions 
Define $\omega_{t,\tau}=\sum_{s=0}^{\infty}\frac{w_{s,\tau}}{\prod_{j}^{s}R_{j}}$, then $$\sum_{s=t}^{\infty}\frac{c_{s,\tau}}{\prod_{j}^{s}R_{j}}=a_{t,\tau}+\omega_{t,\tau}$$

now the LHS using the Euler becomes $$\frac{c_{t,\tau}}{R_{t}}+\frac{c_{t+1,\tau}}{R_{t}R_{t+1}}+\frac{c_{t+2,\tau}}{R_{t}R_{t+1}R_{t+2}}+...$$ $$\frac{c_{t,\tau}}{R_{t}}+\frac{\beta\left(1-\nu\right)R_{t+1}c_{t,\tau}}{R_{t}R_{t+1}}+\frac{\left(\beta\left(1-\nu\right)\right)^{2}R_{t+1}R_{t+2}c_{t,\tau}}{R_{t}R_{t+1}R_{t+2}}+...$$ $$\frac{c_{t,\tau}}{R_{t}}\sum_{t=0}^{\infty}\left(\beta\left(1-\nu\right)\right)^{t}$$



## Aggregation of individual conditions 

Finally we get $$c_{t,\tau}=\left(1-\beta\left(1-\nu\right)\right)R_{t}\left(a_{t,\tau}+\omega_{t,\tau}\right)$$ notice that $R_{t}$ appears even with log utility. This is because $a_{t}$ is the stock at the beginning of the period (again something that does not appear in continous time). Using $\frac{c_{t,\tau}}{R_{t}}=\hat{c}_{t,\tau}$to express $c_{t}$ and$a_{t}$ at the same value time $$\hat{c}_{t,\tau}=\left(1-\beta\left(1-\nu\right)\right)\left(a_{t,\tau}+\omega_{t,\tau}\right)$$ that does not depend on the interest rate. And Aggregate $$C_{t}=\left(1-\beta\left(1-\nu\right)\right)R_{t}\left(A_{t}+\Omega_{t}\right)$$



## Aggregate dynamics  

Divide by $L_{t}$the aggregate equations and use $a_{t}=k_{t}$in equilibirum together with the foc of the firms to substitute for prices $$\frac{k_{t+1}\left(1+n-\nu\right)}{\left(1-\nu\right)}=\left(1+\frac{\nu}{\left(1-\nu\right)}\right)k_{t}-c_{t}+f\left(k_{t}\right)$$ $$c_{t}=\left(1-\beta\left(1-\nu\right)\right)\left(1+f'\left(k_{t}\right)+\frac{\nu}{\left(1-\nu\right)}\right)\left(k_{t}+\omega_{t}\right)$$

where $\omega_{t}$ depends an the whole path of future $k_{t}$.



## Steady State 

Use the fact that $$\omega=\left(f\left(k\right)-kf'\left(k\right)\right)\frac{1+f'\left(k\right)+\frac{\nu}{\left(1-\nu\right)}}{f'\left(k\right)+\frac{\nu}{\left(1-\nu\right)}}$$ then

$$c=f\left(k\right)-\frac{\left(n-\nu\right)}{\left(1-\nu\right)}k$$ $$c=\left(1-\beta\left(1-\nu\right)\right)\left(1+f'\left(k\right)+\frac{\nu}{\left(1-\nu\right)}\right)\left(k+\omega\right)$$



## Conclusion 

The algebra is cumbersome, and not all the results are transparent. In the process we have learned that many difficulties were coming from the discrete time choice.
